# Import necessary libraries

In [1]:
import numpy as np
from scipy.fftpack import dct, idct
from PIL import Image
import pickle
from io import BytesIO
import matplotlib.pyplot as plt
import heapq
from collections import Counter, defaultdict
import os
from bitstring import BitStream
import io
import gzip


# Function to compute DCT and inverse DCT

In [2]:
# DCT and inverse DCT for 8x8 blocks
def compute_dct(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')

def compute_idct(block):
    return idct(idct(block.T, norm='ortho').T, norm='ortho')

# Define the Quantization Matrix

In [3]:
# Standard JPEG quantization matrix
quantization_matrix = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])

# Load the image

In [4]:
def load_image(filepath):
    image = Image.open(filepath).convert('L')   #read the image as grayscale
    return np.array(image)

# Function for Huffman encoding and decoding

In [5]:
# Huffman Tree generation
def build_huffman_tree(data):
    frequency = Counter(data)
    heap = [[weight, [symbol, ""]] for symbol, weight in frequency.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    return {symbol: code for symbol, code in heapq.heappop(heap)[1:]}

def huffman_encode(data, huffman_table):
    return ''.join(huffman_table[val] for val in data)

def huffman_decode(encoded_data, huffman_table):
    inverse_table = {code: symbol for symbol, code in huffman_table.items()}
    decoded_data, current_code = [], ""
    for bit in encoded_data:
        current_code += bit
        if current_code in inverse_table:
            decoded_data.append(inverse_table[current_code])
            current_code = ""
    return decoded_data

# Functions for Quantization

In [6]:

def quantize_block(dct_block, quant_matrix, quality_factor):
    '''Quantize the DCT block'''
    # Ensure quality_factor is within valid bounds
    quality_factor = max(1, min(100, quality_factor))

    #find the scale acc to quality factor 
    if quality_factor < 50:
      scale = 5000 / quality_factor
    else:
      scale = 200 - 2 * quality_factor

    #find the adjusted matrx
    adjusted_matrix = np.floor((quantization_matrix * scale + 50) / 100)
    adjusted_matrix[adjusted_matrix == 0] = 1
    # Quantize the DCT block
    
    quantized_block = np.round(dct_block / adjusted_matrix).astype(int)
    # print(adjusted_matrix)
    return quantized_block, adjusted_matrix

def dequantize_block(quantized_block, adjusted_matrix):
    '''dequantize the block'''
    return (quantized_block * adjusted_matrix).astype(float)

# ZigZag order

In [7]:
def zigzag_order(block):
    '''get zigzag order of the block'''
    rows, cols = block.shape
    solution = [[] for i in range(rows + cols - 1)]

    for row in range(rows):
        for col in range(cols):
            sum = row + col
            if(sum % 2 == 0):
                #add at beginning
                solution[sum].insert(0, block[row][col])
            else:
                #add at end of the list
                solution[sum].append(block[row][col])

    return np.array([item for sublist in solution for item in sublist])
def inverse_zigzag_order(zigzag_values, block_size=8):
    '''get inverse zigzag order'''
    matrix = np.zeros((block_size, block_size), dtype=int)
    index = 0

    for i in range(block_size + block_size - 1):
        if i % 2 == 0:  # Even diagonal (up-right)
            # Moving up-right with decrementing row and incrementing column
            for j in range(min(i + 1, block_size)):
                row = i - j
                col = j
                if row >= 0 and row < block_size and col >= 0 and col < block_size:  # Check matrix boundaries
                    matrix[row, col] = zigzag_values[index]
                    index += 1
        else:  # Odd diagonal (down-left)
            # Moving down-left with incrementing row and decrementing column
            for j in range(min(i + 1, block_size)):
                col = i - j
                row = j
                if row >= 0 and row < block_size and col >= 0 and col < block_size:
                    matrix[row, col] = zigzag_values[index]
                    index += 1

    return matrix

# Run Length Encoding

In [8]:
def run_length_encoding(zigzag_values,huffman_table):
    encoded_values = []
    run_length = 0

    for value in zigzag_values:
        if value == 0:
            run_length += 1
        else:
            encoded_values.append((np.uint8(run_length),huffman_table[value]))  # Store run-length, size, and value
            run_length = 0  # Reset run-length

    # Store trailing zeros count in the EOB marker:
    encoded_values.append((np.uint8(run_length),-1))  # EOB with trailing zeros count
    # encoded_values.append((0, 0, 'EOB'))
    return encoded_values

def run_length_decoding(encoded_values):
    decoded_values = []
    trailing_zeros_count = 0  # Initialize trailing zeros count

    for run_length , value in encoded_values:
        if value == -1:  # Check for EOB marker
            trailing_zeros_count = value  # Get trailing zeros count from EOB
            break  # Stop decoding at the EOB marker
        else:
            decoded_values.extend([0] * run_length)  # Add zeros for the run-length
            decoded_values.append(value)  # Append the actual value

    # Add trailing zeros (if any) after decoding:
    decoded_values.extend([0] * trailing_zeros_count)

    return decoded_values

# Padding

In [9]:
def pad_image(image, block_size=8):
    """Pad the image to make its dimensions a multiple of block_size."""
    height, width = image.shape
    pad_height = (block_size - (height % block_size)) % block_size
    pad_width = (block_size - (width % block_size)) % block_size
    padded_image = np.pad(image, ((0, pad_height), (0, pad_width)), mode='constant')
    return padded_image, height, width

# Compress Image JPEG

In [10]:
def compress_image(filepath, quality_factor):
    image = load_image(filepath)
    height, width = image.shape
    compressed_data = []
    dc_diff_values = []
    ac_values = []

    adjusted_matrix = None
    quantized_blocks = []
    dct_blocks = []

    #pad the image so that both row and column are nearest multiple 8
    padded_image, original_height, original_width = pad_image(image, 8)
    padded_height, padded_width = padded_image.shape
    
    # Divide into 8x8 blocks, apply DCT and quantize
    previous_dc = 0
    #store the orignal size of image as well
    for i in range(0, padded_height, 8):
        for j in range(0, padded_width, 8):
            dct_blocks.append(padded_image[i:i+8, j:j+8])
            block = padded_image[i:i+8, j:j+8]
            dct_block = compute_dct(block)
   
            quantized_block, adjusted_matrix = quantize_block(dct_block, quantization_matrix, quality_factor)
            quantized_blocks.append(quantized_block)

            # DC and AC separation
            dc_diff = quantized_block[0, 0] - previous_dc
            previous_dc = quantized_block[0, 0]
            dc_diff_values.append(dc_diff)
            ac_values.extend(zigzag_order(quantized_block)[1:])

    # Build Huffman trees
    dc_huffman_table = build_huffman_tree(dc_diff_values)
    ac_huffman_table = build_huffman_tree([val for val in ac_values if val != 0])
   
    # Compress each block
    previous_dc = 0
    for i in range(0, padded_height, 8):
        for j in range(0, padded_width, 8):
            block = padded_image[i:i+8, j:j+8]
            dct_block = compute_dct(block)
            quantized_block, adjusted_matrix = quantize_block(dct_block, quantization_matrix, quality_factor)

            # Encode DC
            dc_diff = quantized_block[0, 0] - previous_dc
            previous_dc = quantized_block[0, 0]
            dc_encoded = dc_huffman_table[dc_diff]

            # Encode AC with run-length, size, and Huffman code
            zigzag_ac = zigzag_order(quantized_block)[1:]
            ac_encoded = run_length_encoding(zigzag_ac,ac_huffman_table)
            compressed_data.append((dc_encoded, ac_encoded))

    header = {
        'file_size': (height, width),
        'dc_huffman_table': dc_huffman_table,
        'ac_huffman_table': ac_huffman_table,
        'adjusted_matrix': adjusted_matrix
    }

    compressed_stream = BytesIO()
    with gzip.GzipFile(fileobj=compressed_stream, mode='wb') as f:
        pickle.dump(header, f)
        pickle.dump(compressed_data, f)

    return compressed_stream.getvalue(), dc_huffman_table, ac_huffman_table

# Decompress Image JPEG

In [11]:
def decompress_image(compressed_data):

    compressed_stream = BytesIO(compressed_data)

    # Open the gzip file in read mode
    with gzip.GzipFile(fileobj=compressed_stream, mode='rb') as f:
        # Load the header and compressed data from the stream
        header = pickle.load(f)
        compressed_blocks = pickle.load(f)
    
    # Extract header information
    original_height, original_width = header['file_size']  # Extract original dimensions
    dc_huffman_table = header['dc_huffman_table']
    ac_huffman_table = header['ac_huffman_table']
    adjusted_matrix = header['adjusted_matrix']
    
    # Initialize decompressed image with padded size
    blocks_per_row = int(np.sqrt(len(compressed_blocks)))  
    block_size = 8
    decompressed_image = np.zeros((original_height, original_width), dtype=np.uint8)
    decompressed_image = pad_image(decompressed_image, block_size)[0]
    padded_height, padded_width = decompressed_image.shape

    # Initialize previous DC coefficient to 0
    previous_dc = 0
    idx = 0
    dequantized_blocks = []
    inverse_dct = []

    # Decompress each block
    for i in range(0, padded_height, block_size):
        for j in range(0, padded_width, block_size):
            dc_encoded, ac_encoded = compressed_blocks[idx]

            # Decode DC coefficient
            dc_diff = huffman_decode(dc_encoded, dc_huffman_table)[0]
            dc_coeff = previous_dc + dc_diff
            previous_dc = dc_coeff

            # Decode AC coefficients
            ac_decoded = []
            trailing_zeros = 0
            for run_length, value in ac_encoded:
                if value == -1:
                    trailing_zeros = run_length
                    break
                else:
                    zeros = [0] * run_length
                    decoded_value = huffman_decode(value, ac_huffman_table)[0]
                    ac_decoded.extend(zeros + [decoded_value])

            # Pad decoded AC coefficients with zeros if necessary and add trailing zeros
            ac_decoded.extend([0] * (63 - len(ac_decoded)))
            ac_decoded.extend([0] * trailing_zeros)

            # Insert DC coefficient at the beginning of ac_decoded
            ac_decoded.insert(0, dc_coeff)

            # Reconstruct the 8x8 block using inverse zigzag
            quantized_block = inverse_zigzag_order(ac_decoded, block_size=8)

            # Dequantize the block
            dct_block = dequantize_block(quantized_block, adjusted_matrix)
            dequantized_blocks.append(dct_block)

            # Apply inverse DCT
            decompressed_block = compute_idct(dct_block)
            inverse_dct.append(decompressed_block)

            # Clip pixel values to valid range and assign to decompressed image
            decompressed_image[i:i + 8, j:j + 8] = np.clip(decompressed_block, 0, 255)
            idx += 1

    # Crop the decompressed image to its original size
    cropped_image = decompressed_image[:original_height, :original_width]
    return cropped_image

# Showtime

In [12]:
def calculate_rmse(original_image, decompressed_image):
    """Calculates the RMSE between two images."""
    return np.sqrt(np.mean((original_image - decompressed_image)**2))

def calculate_bpp(compressed_data, image_shape):
    """Calculates the BPP of a compressed image."""
    compressed_size_bits = len(compressed_data)*8 # Size in bits
    num_pixels = image_shape[0] * image_shape[1]
    return compressed_size_bits / num_pixels


def analyze_images(folder_path, quality_factors, output_folder):
    """Simulates different quality factors and plots RMSE vs BPP, saving the plots to output_folder."""
    
    min_rmse = np.inf
    max_rmse = -np.inf
    min_rmse_image = None
    max_rmse_image = None

    for quality_factor in quality_factors:
        rmse_values = []
        bpp_values = []
        compression_rates = []
        image_labels = []  # To track filenames for compression rate plot

        for filename in os.listdir(folder_path):
            if filename.endswith(('.png', '.jpg', '.jpeg', '.JPG')):  # Check for image files
                filepath = os.path.join(folder_path, filename)
                original_image = load_image(filepath)  # Load image using your function

                # Compress and decompress the image
                compressed_data, _, _ = compress_image(filepath, quality_factor)
                decompressed_image = decompress_image(compressed_data)

                # Calculate RMSE and BPP
                rmse = calculate_rmse(original_image, decompressed_image)
                bpp = calculate_bpp(compressed_data, original_image.shape)
                original_size = original_image.nbytes  # Get original file size in bytes
                compressed_size = len(compressed_data)
                compression_rate = original_size / compressed_size

                # Store the values
                rmse_values.append(rmse)
                bpp_values.append(bpp)
                compression_rates.append(compression_rate)
                image_labels.append(filename)

                # Save the comparison image
                comparison_image_path = os.path.join(output_folder, f"comparison_quality_{quality_factor}_{filename}")
                # show_images(original_image, decompressed_image, quality_factor, comparison_image_path)

                # Track min and max RMSE images
                if rmse < min_rmse:
                    min_rmse = rmse
                    min_rmse_image = (original_image, decompressed_image, quality_factor, filename)
                
                if rmse > max_rmse:
                    max_rmse = rmse
                    max_rmse_image = (original_image, decompressed_image, quality_factor, filename)

        # Save RMSE vs BPP plot for the current quality factor
        plt.figure(figsize=(8, 6))
        plt.scatter(bpp_values, rmse_values, marker='o', color='blue', label=f'Quality Factor: {quality_factor}')
        plt.xlabel("BPP (Bits per Pixel)")
        plt.ylabel("RMSE (Root Mean Squared Error)")
        plt.title(f"RMSE vs BPP (Quality Factor: {quality_factor})")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        
        # Save the plot as an image file
        rmse_bpp_filename = os.path.join(output_folder, f"RMSE_vs_BPP_QF_{quality_factor}.png")
        plt.savefig(rmse_bpp_filename)
        plt.close()  # Close the plot to avoid overlapping with the next one

        # Save Compression Rate plot for the current quality factor
        plt.figure(figsize=(8, 6))
        plt.bar(image_labels, compression_rates, color='orange')
        plt.xlabel("Image")
        plt.ylabel("Compression Rate")
        plt.title(f"Compression Rate per Image (Quality Factor: {quality_factor})")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Save the plot as an image file
        compression_rate_filename = os.path.join(output_folder, f"Compression_Rate_QF_{quality_factor}.png")
        plt.savefig(compression_rate_filename)
        plt.close()  # Close the plot to avoid overlapping with the next one

        print(f"Saved plots for Quality Factor {quality_factor} in {output_folder}")

    # Save images with minimum and maximum RMSE
    if min_rmse_image:
        min_image_path = os.path.join(output_folder, f"min_rmse_image_{min_rmse:.2f}.png")
        show_images(min_rmse_image[0], min_rmse_image[1], min_rmse_image[2], min_image_path)
        print(f"Saved image with minimum RMSE ({min_rmse:.2f}) at {min_rmse_image[3]}")
        
    if max_rmse_image:
        max_image_path = os.path.join(output_folder, f"max_rmse_image_{max_rmse:.2f}.png")
        show_images(max_rmse_image[0], max_rmse_image[1], max_rmse_image[2], max_image_path)
        print(f"Saved image with maximum RMSE ({max_rmse:.2f}) at {max_rmse_image[3]}")

# Function to show and save images
def show_images(original_image, decompressed_image, quality_factor, save_path):
    """
    Displays and saves a comparison of the original and decompressed images.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Original image
    axes[0].imshow(original_image, cmap='gray')
    axes[0].set_title("Original Image")
    axes[0].axis('off')
    
    # Decompressed image
    axes[1].imshow(decompressed_image, cmap='gray')
    axes[1].set_title(f"Decompressed Image (Quality Factor: {quality_factor})")
    axes[1].axis('off')
    
    # Save the figure
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()  # Free memory




# Showtime

In [ ]:
# Simulated list of quality factors
quality_factors = [10, 50, 100]  # Simulate quality factors

# Filepath of the image to process
filepath = './dataset/urban100/img_010_SRF_2_LR.png'

# Folder to save results
save_folder = './comparison_images'
os.makedirs(save_folder, exist_ok=True)  # Ensure the folder exists

# Process the image for each quality factor
for quality_factor in quality_factors:
    # Load the original image
    original_image = load_image(filepath)
    
    # Compress and decompress the image
    compressed_data, _, _ = compress_image(filepath, quality_factor)
    decompressed_image = decompress_image(compressed_data)
    # Save the comparison image
    save_path = os.path.join(save_folder, f"comparison_quality_{quality_factor}.png")
    show_images(original_image, decompressed_image, quality_factor, save_path)

analyze_images('./dataset/urban100', quality_factors,'./Results')